In [1]:
import warnings
import numpy as np
import pandas as pd
from utils import *
from tqdm import tqdm
from deep_translator import GoogleTranslator
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')
path = '../../../datasets/garanti-bbva-data-camp/clean_language.csv'

In [2]:
def load_languages(path: str, size: int = 8):

    df_ = pd.read_csv(path)
    df_ = df_.drop_duplicates()

    vectorizer = CountVectorizer(max_features=size, ngram_range=(1, 1))

    return (
        pd.DataFrame(
            vectorizer.fit_transform(df_["language"]).toarray(),
            columns=[f"language_{str(f)}" for f in vectorizer.get_feature_names()],
        )
        .assign(user_id=lambda x: df_["user_id"].tolist())
        .groupby(by="user_id", as_index=False)
        .sum()
        .merge(
            df_.groupby(by="user_id", as_index=False).agg(
                total_languages=("language", "nunique")
            ),
            on=["user_id"],
            how="left",
        )
    )

In [13]:
#lang_df = load_languages('../../../datasets/garanti-bbva-data-camp/clean_language.csv', 55)
#
#lang_df

In [12]:
#[col for col in lang_df.columns if ',' in col]

In [1]:
#language_grouped = (
#        df[["user_id"]]
#        .drop_duplicates()
#        .merge(
#            df.dropna(subset=["language"])
#            .groupby(by="user_id", as_index=False)
#            .agg(
#                {
#                    "language": lambda x: [
#                        i for i in x.unique()
#                    ]
#                }
#            ),
#            on=["user_id"],
#            how="left",
#        )
#        .assign(language=lambda x: x.language.fillna("").apply(list))
#    )
#
#
#language_grouped

In [18]:
#grouped = df[["user_id"]].drop_duplicates().merge(
#            df.dropna(subset=["language"]).groupby(by="user_id", as_index=False).agg(
#                {
#                    "language": lambda x: [
#                        s.strip() for s in ", ".join(x.unique()).split(",")
#                    ]
#                }
#            ),
#            on=["user_id"],
#            how="left",
#        ).assign(language=lambda x: x.language.fillna("").apply(list))
#
#grouped["total_languages"] = grouped["language"].apply(lambda x: len(x))
#print(f"grouped shape: {grouped.shape}")

In [1]:
#def load_languages(path: str, size: int = 8, seperator: str = "language"):
#
#    df_ = pd.read_csv(path)
#    df_ = df_.drop_duplicates()
#
#    vectorizer = CountVectorizer(
#        max_features=size, stop_words=stopwords.words("english"), ngram_range=(1, 1)
#    )
#
#    total_langs_df = df_.groupby(by="user_id", as_index=False).agg(
#        total_languages=(
#            "language",
#            lambda x: len([s.strip() for s in ", ".join(x.unique()).split(",")]),
#        )
#    )
#
#    lang_grouped_df = df_.groupby(by="user_id", as_index=False).agg(
#        {"language": lambda x: " ".join(x.unique())}
#    )
#
#    return (
#        pd.DataFrame(
#            vectorizer.fit_transform(lang_grouped_df["language"]).toarray(),
#            columns=[f"language_{str(f)}" for f in vectorizer.get_feature_names()],
#        )
#        .assign(user_id=lang_grouped_df["user_id"])
#        .merge(total_langs_df, on=["user_id"], how="left")
#    )


In [57]:
def load_languages_v2(
    path: str, size: int = 8, seperator: str = "language"
) -> pd.DataFrame:

    df_ = pd.read_csv(path)
    df_ = fix_languages(df_)
    df_["language"] = df_["language"].apply(lambda x: x.upper().strip())
    df_["language"] = df_["language"].apply(lambda x: translation(str(x)))
    total_langs_df = df_.groupby(by="user_id", as_index=False).agg(
        total_langs=(
            "language",
            lambda x: len([s.strip() for s in ", ".join(x.unique()).split(",")]),
        )
    )

    vectorizer = CountVectorizer(
        max_features=size, stop_words=stopwords.words("english"), ngram_range=(1, 1)
    )

    lang_grouped_df = df_.groupby(by="user_id", as_index=False).agg(
        {"language": lambda x: " ".join(x.unique())}
    )

    return (
        pd.DataFrame(
            vectorizer.fit_transform(lang_grouped_df["language"]).toarray(),
            columns=[f"language_{str(f)}" for f in vectorizer.get_feature_names()],
        )
        .assign(user_id=lang_grouped_df["user_id"])
        .merge(total_langs_df, on=["user_id"], how="left")
    )

In [44]:
#df = pd.read_csv(path)
#print(df.shape)
#df.head()

In [45]:
#df = df.drop_duplicates()
#print(df.shape)